In [1]:
import uproot
import sys, os
import pandas as pd
import numpy as np
sys.path.append('../PIDGen_PIDCalib_MVA/')
from Add_MVA import AddBDTinfo
from AddSweights import *

filedir = '/disk/lhcb_data2/RLcMuonic2016/'

Welcome to JupyROOT 6.18/04


In [3]:
#varsON = ['Lb_L0Global_TIS','Lb_L0HadronDecision_TOS','Lc_Hlt1TrackMVADecision_TOS',
#          'Lc_Hlt1TwoTrackMVADecision_TOS','Lb_Hlt2XcMuXForTauB2XcMuDecision_TOS',
#          'Lb_Hlt2XcMuXForTauB2XcFakeMuDecision_TOS','Lc_M','p_ProbNNp','p_ProbNNk','mu_PID*','*_P',
#          '*_PT','nTracks','runNumber','eventNumber','Lb_ISOLATION_*','mu_PX','mu_PY','mu_PZ','mu_ID',
#          'Lc_PX','Lc_PY','Lc_PZ']
df_list=[]
for df in uproot.pandas.iterate(filedir+'Data/Lb_DataSS_MagUp.root','tupleout/DecayTree', entrysteps=500000):
#                            varsON, entrysteps=500000):
    df_list.append(df)

KeyboardInterrupt: 

In [ ]:
df = df_list[0].copy()
df.head()

In [2]:
df.to_csv('df0.csv',index=False)

NameError: name 'df' is not defined

In [3]:
df = pd.read_csv("df0.csv") 
df.head()

,Lb_L0Global_TIS,Lb_L0HadronDecision_TOS,Lc_Hlt1TrackMVADecision_TOS,Lc_Hlt1TwoTrackMVADecision_TOS,Lb_Hlt2XcMuXForTauB2XcMuDecision_TOS,Lb_Hlt2XcMuXForTauB2XcFakeMuDecision_TOS,Lc_M,p_ProbNNp,p_ProbNNk,mu_PIDmu,...,Lb_ISOLATION_IsMuon3,Lb_ISOLATION_NNghost3,Lb_ISOLATION_OldFDCHI2,mu_PX,mu_PY,mu_PZ,mu_ID,Lc_PX,Lc_PY,Lc_PZ
0,True,False,False,True,True,False,2293.895486,0.139599,0.240121,-6.083391,...,0.0,0.014878,6433.632000,-201.19,158.31,4328.02,13,-1960.88,-2152.35,63309.98
1,False,False,True,True,True,False,2241.863511,0.630536,0.040897,7.032762,...,0.0,0.764885,16023.387000,1129.55,-1119.00,16550.21,13,6361.34,-5894.45,75556.70
2,False,False,True,True,True,False,2272.953145,0.056319,0.993643,9.509515,...,0.0,0.001384,106.833650,-57.38,204.72,5553.79,-13,4257.29,5955.58,99383.79
3,True,False,True,True,True,False,2249.818104,0.021579,0.999310,2.282870,...,0.0,0.013565,48.791176,122.23,75.46,4763.67,-13,7568.78,-2914.19,66231.56
4,False,False,True,True,True,False,2320.249556,0.495502,0.988775,5.231485,...,0.0,0.781538,4482.420000,1788.88,-286.99,29836.72,13,3292.81,1116.76,50034.20


In [4]:
%%time

def L0TriggerData(df):
    df['L0'] = False
    df.loc[df.Lb_L0Global_TIS | df.Lb_L0HadronDecision_TOS,'L0'] = True
    return df

def HLT1TriggerData(df):
    df['HLT1'] = False
    df.loc[df.Lc_Hlt1TrackMVADecision_TOS | df.Lc_Hlt1TwoTrackMVADecision_TOS,'HLT1'] = True
    return df

def HLT2TriggerData(df,dtype):
    df['HLT2'] = False
    if dtype =='Data' or dtype == 'DataSS':
        df.loc[df.Lb_Hlt2XcMuXForTauB2XcMuDecision_TOS,'HLT2'] = True
    if dtype =='FakeMu' or dtype=='FakeMuSS':
        df.loc[df.Lb_Hlt2XcMuXForTauB2XcFakeMuDecision_TOS,'HLT2'] = True
    return df

def TriggerData(df):
    df['Trigger'] = False
    df.loc[df.L0 & df.HLT1 & df.HLT2,'Trigger'] = True
    return df

df1 = df.copy()
df1 = L0TriggerData(df1)
df1 = HLT1TriggerData(df1)
df1 = HLT2TriggerData(df1,'DataSS')
df1 = TriggerData(df1)
df1.head()
df1.loc[df1.Trigger==True].shape

CPU times: user 116 ms, sys: 103 ms, total: 219 ms
Wall time: 217 ms


(266060, 82)

In [5]:
%%time

def LcMassCut(df):
    df['LcMass'] = False
    df.loc[(df.Lc_M>2230) & (df.Lc_M<2330),'LcMass'] = True
    return df

df1 = LcMassCut(df1)
df1.loc[(df1.Trigger==True) & (df1.LcMass==True)].shape
#df1.loc[(df1.LcMass==True)].shape

CPU times: user 73.3 ms, sys: 16.3 ms, total: 89.6 ms
Wall time: 87.1 ms


(203951, 83)

In [6]:
%%time

def ApplyMuCuts(df,dtype):
    if dtype=='Data'or dtype=='DataSS':
        df['MuCuts'] = False
        df.loc[(df.mu_PIDmu>2) & ((df.mu_PIDmu -df.mu_PIDK)>2) & ((df.mu_PIDmu -df.mu_PIDp)>2),'MuCuts']=True
    else:
        df['MuCuts'] = True
    return df

df1 = ApplyMuCuts(df1,'DataSS')
df1.loc[(df1.Trigger==True) & (df1.LcMass==True) & (df1.MuCuts==True)].shape
#df1.head()

CPU times: user 41.3 ms, sys: 12.4 ms, total: 53.7 ms
Wall time: 201 ms


(57141, 84)

In [7]:
%%time
def ApplyPIDCalibCuts(df):
    df['PIDCalib'] = False
    df.loc[df.nTracks.between(0,700)&df.mu_P.between(0,200000)&df.mu_PT.between(0,60000)&
          df.pi_P.between(0,200000)&df.pi_PT.between(0,60000)&
          df.p_P.between(0,200000)&df.p_PT.between(0,60000)&
          df.K_P.between(0,200000)&df.K_PT.between(0,60000),'PIDCalib'] = True
    return df
df1 = ApplyPIDCalibCuts(df1)
df1.loc[(df1.Trigger==True) & (df1.LcMass==True) & (df1.MuCuts==True) & (df1.PIDCalib==True)].shape
#df1.head()

CPU times: user 52.9 ms, sys: 3.06 ms, total: 56 ms
Wall time: 54.2 ms


(56926, 85)

In [8]:
%%time

def GetFinalPreselection(df):
    df['Preselection'] = False
    df.loc[df.Trigger & df.LcMass & df.MuCuts &df.PIDCalib,'Preselection'] = True
    return df

df1 = GetFinalPreselection(df1)
df1.loc[df1.Preselection==True].shape
#df1.head()

CPU times: user 45.8 ms, sys: 3.33 ms, total: 49.2 ms
Wall time: 47.2 ms


(56926, 86)

In [9]:
def LoadBDTdf(dtype,polarity):
    dflist_bdt=[]
    ifname = filedir+'Data/Lb_'+dtype+'_'+polarity+'.root'
    bdtfname = ifname[0:-5]+'_MVA.root'
    if os.path.isfile(bdtfname):
        print('BDT file already created')
    else:
        print()
        print('>>>   Creating file with BDT variable')
        print()
        AddBDTinfo(ifname, 'tupleout/DecayTree', bdtfname, 'Data',
                   pickled_model_path = '../PIDGen_PIDCalib_MVA/xgb_reg.pkl')
    for df in uproot.pandas.iterate(bdtfname,'DecayTree',
                            'bdt', entrysteps=500000):
        dflist_bdt.append(df)
    return dflist_bdt
dflist_bdt = LoadBDTdf('DataSS','MagUp')
df_bdt = dflist_bdt[0].copy()
df_bdt.head()

BDT file already created


,bdt
0,0.001160
1,0.273441
2,0.011024
3,0.008728
4,0.223288


In [10]:
df2 = df1.merge(df_bdt['bdt'],left_index=True, right_index=True)
df2.head()

,Lb_L0Global_TIS,Lb_L0HadronDecision_TOS,Lc_Hlt1TrackMVADecision_TOS,Lc_Hlt1TwoTrackMVADecision_TOS,Lb_Hlt2XcMuXForTauB2XcMuDecision_TOS,Lb_Hlt2XcMuXForTauB2XcFakeMuDecision_TOS,Lc_M,p_ProbNNp,p_ProbNNk,mu_PIDmu,...,Lc_PZ,L0,HLT1,HLT2,Trigger,LcMass,MuCuts,PIDCalib,Preselection,bdt
0,True,False,False,True,True,False,2293.895486,0.139599,0.240121,-6.083391,...,63309.98,True,True,True,True,True,False,True,False,0.001160
1,False,False,True,True,True,False,2241.863511,0.630536,0.040897,7.032762,...,75556.70,False,True,True,False,True,True,True,False,0.273441
2,False,False,True,True,True,False,2272.953145,0.056319,0.993643,9.509515,...,99383.79,False,True,True,False,True,True,True,False,0.011024
3,True,False,True,True,True,False,2249.818104,0.021579,0.999310,2.282870,...,66231.56,True,True,True,True,True,True,True,True,0.008728
4,False,False,True,True,True,False,2320.249556,0.495502,0.988775,5.231485,...,50034.20,False,True,True,False,True,True,True,False,0.223288


In [11]:
%%time

BDTcut=0.7
def PassBDT(df,BDTcut):
    df['PassBDT'] = False
    df.loc[df.bdt>BDTcut,'PassBDT']=True
    return df

df2 = PassBDT(df2,BDTcut)
df2.loc[(df2.Preselection) & (df2.PassBDT)].shape

CPU times: user 93.1 ms, sys: 109 ms, total: 202 ms
Wall time: 200 ms


(4471, 88)

In [12]:
%%time

def RemoveDDstar(df):
    df['NoDDstar'] = False
    df.loc[((df.p_ProbNNp - df.p_ProbNNk)>0),'NoDDstar']=True
    return df

def FinalSelection(df):
    df['FinalSel']= False
    df.loc[df.Preselection & df.PassBDT & df.NoDDstar,'FinalSel']=True
    return df

df2 = RemoveDDstar(df2)
df2 = FinalSelection(df2)
df2.loc[df2.FinalSel].shape
#df2.head()


CPU times: user 30.5 ms, sys: 3.64 ms, total: 34.2 ms
Wall time: 28 ms


(4205, 90)

In [13]:
df2.loc[(df2.NoDDstar==True) & (df2.PassBDT==True)& (df2.Preselection==True)].shape

(4205, 90)

In [14]:
%%time
ISOBDTcut =0.35
ISOBDT2cut=0.2

#Masses pi, K, p, mu, Lc
m_pi = 139.57018 #+/- 0.00035 MeV (PDG)
m_K = 493.677 #+/- 0.016 MeV (PDG)
m_p = 938.272081 #+/- 0.000006 MeV (PDG)
m_mu = 105.6583745 #+/- 0.0000024 MeV (PDG)
m_Lc = 2286.46 #+/- 0.14 MeV (PDG)

def CheckIfIsKenriched(df):
    df1 = df[(df.Lb_ISOLATION_BDT>ISOBDTcut) & (df.Lb_ISOLATION_BDT2>ISOBDT2cut)].copy()
    df1['E1pi'] = np.sqrt(df1["Lb_ISOLATION_PX"]**2 + df1['Lb_ISOLATION_PY']**2 + df1['Lb_ISOLATION_PZ']**2 + m_pi**2)
    df1['E2pi'] = np.sqrt(df1["Lb_ISOLATION_PX2"]**2 + df1['Lb_ISOLATION_PY2']**2 + df1['Lb_ISOLATION_PZ2']**2 + m_pi**2)
    df1['ELc'] =np.sqrt(df1['Lc_PX']**2+df1['Lc_PY']**2+df1['Lc_PZ']**2 + m_Lc**2)
    df1['pLc12_x'] = df1['Lc_PX']+df1["Lb_ISOLATION_PX"]+df1["Lb_ISOLATION_PX2"]
    df1['pLc12_y'] = df1['Lc_PY']+df1["Lb_ISOLATION_PY"]+df1["Lb_ISOLATION_PY2"]
    df1['pLc12_z'] = df1['Lc_PZ']+df1["Lb_ISOLATION_PZ"]+df1["Lb_ISOLATION_PZ2"]
    df1['mLc12'] = np.sqrt((df1['ELc']+df1['E1pi']+df1['E2pi'])**2 -
                           (df1['pLc12_x']**2+df1['pLc12_y']**2+df1['pLc12_z']**2))
    df1['muCharge'] = -df1['mu_ID']/13
    df1['PIDdiff'] =df1['Lb_ISOLATION_PIDp'] - df1['Lb_ISOLATION_PIDK']
    df1['PIDdiff2'] =df1['Lb_ISOLATION_PIDp2'] - df1['Lb_ISOLATION_PIDK2']
    df1['m1']= m_pi
    df1['m2']= m_pi
    df1.loc[(df1.Lb_ISOLATION_PIDK>4.)&((df1.Lb_ISOLATION_CHARGE==-df1.muCharge)|
                                        ((df1.Lb_ISOLATION_CHARGE==df1.muCharge) 
                                         & (df1.PIDdiff<0))),'m1']= m_K
    df1.loc[(df1.Lb_ISOLATION_PIDK2>4.)&((df1.Lb_ISOLATION_CHARGE2==-df1.muCharge)|
                                        ((df1.Lb_ISOLATION_CHARGE2==df1.muCharge) 
                                         & (df1.PIDdiff2<0))),'m2']= m_K
    #df1['m1'] = df1.apply(lambda x: m_K if (x.Lb_ISOLATION_PIDK>4.)&((x.Lb_ISOLATION_CHARGE==-x.muCharge)|((x.Lb_ISOLATION_CHARGE==x.muCharge) & (x.PIDdiff<0))) else m_pi, axis=1)
    #df1['m2'] = df1.apply(lambda x: m_K if (x.Lb_ISOLATION_PIDK2>4.)&((x.Lb_ISOLATION_CHARGE2==-x.muCharge)|((x.Lb_ISOLATION_CHARGE2==x.muCharge) & (x.PIDdiff2<0))) else m_pi, axis=1)

    df1['E1'] = np.sqrt(df1["Lb_ISOLATION_PX"]**2 + df1['Lb_ISOLATION_PY']**2 + df1['Lb_ISOLATION_PZ']**2 +df1['m1']**2)
    df1['E2'] = np.sqrt(df1["Lb_ISOLATION_PX2"]**2 + df1['Lb_ISOLATION_PY2']**2 + df1['Lb_ISOLATION_PZ2']**2 +df1['m2']**2)
    df1['Emu'] = np.sqrt(df1.mu_PX**2 + df1.mu_PY**2 + df1.mu_PZ**2 +m_mu**2)
    df1['ETOT'] = df1['ELc']+df1['E1']+df1['E2']+df1['Emu']
    df1['pTOT_x'] = df1['pLc12_x']+df1.mu_PX
    df1['pTOT_y'] = df1['pLc12_y']+df1.mu_PY
    df1['pTOT_z'] = df1['pLc12_z']+df1.mu_PZ
    df1['mTOT'] = np.sqrt(df1.ETOT**2 -(df1.pTOT_x**2+df1.pTOT_y**2+df1.pTOT_z**2))
    df1['isKenriched'] = False
    df1.loc[(df1.mLc12>2700) & (df1.mTOT<5620) & ((df1.m1==m_K) | (df1.m2==m_K)), 'isKenriched']= True 
    df1=df1.drop(df1.loc[:, 'E1pi':'mTOT'].columns,axis=1)
    
    return df1

df1 = CheckIfIsKenriched(df2)
dfT = pd.merge(df2,df1['isKenriched'],how='left',left_index=True, right_index=True)
dfT.loc[dfT['isKenriched'].isnull(),'isKenriched']=False

CPU times: user 535 ms, sys: 428 ms, total: 963 ms
Wall time: 853 ms


In [15]:
dfT.loc[dfT['isKenriched'].isnull(),'isKenriched']=False
dfT['isKenriched']=dfT['isKenriched'].astype('bool')

In [16]:
dfT.loc[(dfT.FinalSel) & (dfT.isKenriched)].shape

(382, 91)

In [17]:
%%time

def CheckIfIsIsolated(df):
    df['isIsolated'] = False
    df.loc[df.Lb_ISOLATION_BDT<ISOBDTcut,'isIsolated']=True
    return df

dfT = CheckIfIsIsolated(dfT)
dfT.loc[(dfT.FinalSel) & dfT.isIsolated==True].shape

CPU times: user 18.4 ms, sys: 641 µs, total: 19.1 ms
Wall time: 17.5 ms


(2021, 92)

In [18]:
dfT

,Lb_L0Global_TIS,Lb_L0HadronDecision_TOS,Lc_Hlt1TrackMVADecision_TOS,Lc_Hlt1TwoTrackMVADecision_TOS,Lb_Hlt2XcMuXForTauB2XcMuDecision_TOS,Lb_Hlt2XcMuXForTauB2XcFakeMuDecision_TOS,Lc_M,p_ProbNNp,p_ProbNNk,mu_PIDmu,...,LcMass,MuCuts,PIDCalib,Preselection,bdt,PassBDT,NoDDstar,FinalSel,isKenriched,isIsolated
0,True,False,False,True,True,False,2293.895486,0.139599,0.240121,-6.083391,...,True,False,True,False,0.001160,False,False,False,True,False
1,False,False,True,True,True,False,2241.863511,0.630536,0.040897,7.032762,...,True,True,True,False,0.273441,False,True,False,False,True
2,False,False,True,True,True,False,2272.953145,0.056319,0.993643,9.509515,...,True,True,True,False,0.011024,False,False,False,False,False
3,True,False,True,True,True,False,2249.818104,0.021579,0.999310,2.282870,...,True,True,True,True,0.008728,False,False,False,False,False
4,False,False,True,True,True,False,2320.249556,0.495502,0.988775,5.231485,...,True,True,True,False,0.223288,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,True,False,True,True,True,False,2271.636965,0.042828,0.998249,5.800219,...,True,True,True,True,0.011298,False,False,False,True,False
499996,False,False,False,False,True,False,2328.887717,0.029709,0.090516,5.794516,...,True,True,True,False,0.005908,False,False,False,False,False
499997,False,False,False,True,True,False,2273.122082,0.014471,0.025393,-1.795917,...,True,False,False,False,0.003769,False,False,False,False,True
499998,True,False,False,False,True,False,2233.107924,0.285419,0.771195,4.822522,...,True,True,True,False,0.001157,False,False,False,False,False


In [19]:
#Create Kenr dataframe
dfKenr =  dfT.copy()
dfKenr = dfKenr.loc[(dfKenr.FinalSel==True) & (dfKenr.isIsolated==False) & (dfKenr.isKenriched==True)]
dfKenr
dfKenr.to_root('Preselected_Kenr.root','DecayTree')

In [24]:
def CheckIfIsLcpipi(df):
    df1 = df[(df.Lb_ISOLATION_BDT>ISOBDTcut) & (df.Lb_ISOLATION_BDT2>ISOBDT2cut) & (df.Lb_ISOLATION_CHARGE==-df.Lb_ISOLATION_CHARGE2) & (df.isKenriched==False)].copy()
    df1['isLcpipi'] = False    
    df1['E1pi'] = np.sqrt(df1["Lb_ISOLATION_PX"]**2 + df1['Lb_ISOLATION_PY']**2 + df1['Lb_ISOLATION_PZ']**2 + m_pi**2)
    df1['E2pi'] = np.sqrt(df1["Lb_ISOLATION_PX2"]**2 + df1['Lb_ISOLATION_PY2']**2 + df1['Lb_ISOLATION_PZ2']**2 + m_pi**2)
    df1['ELc'] =np.sqrt(df1['Lc_PX']**2+df1['Lc_PY']**2+df1['Lc_PZ']**2 + m_Lc**2)
    df1['pLc12_x'] = df1['Lc_PX']+df1["Lb_ISOLATION_PX"]+df1["Lb_ISOLATION_PX2"]
    df1['pLc12_y'] = df1['Lc_PY']+df1["Lb_ISOLATION_PY"]+df1["Lb_ISOLATION_PY2"]
    df1['pLc12_z'] = df1['Lc_PZ']+df1["Lb_ISOLATION_PZ"]+df1["Lb_ISOLATION_PZ2"]
    df1['mLc12'] = np.sqrt((df1['ELc']+df1['E1pi']+df1['E2pi'])**2 -
                           (df1['pLc12_x']**2+df1['pLc12_y']**2+df1['pLc12_z']**2))
    df1.loc[df1['mLc12']<2700,'isLcpipi']= True 
    df1=df1.drop(df1.loc[:, 'E1pi':'pLc12_z'].columns,axis=1)
    return df1

dfLcpipi=dfT.copy()
dfLcpipi1=CheckIfIsLcpipi(dfLcpipi)
dfT = pd.merge(dfLcpipi,dfLcpipi1['isLcpipi'],how='left',left_index=True, right_index=True)
dfT.loc[dfT['isLcpipi'].isnull(),'isLcpipi']=False


In [27]:
dfT.loc[(dfT.FinalSel) & (dfT.isLcpipi)].shape
dfT

,Lb_L0Global_TIS,Lb_L0HadronDecision_TOS,Lc_Hlt1TrackMVADecision_TOS,Lc_Hlt1TwoTrackMVADecision_TOS,Lb_Hlt2XcMuXForTauB2XcMuDecision_TOS,Lb_Hlt2XcMuXForTauB2XcFakeMuDecision_TOS,Lc_M,p_ProbNNp,p_ProbNNk,mu_PIDmu,...,MuCuts,PIDCalib,Preselection,bdt,PassBDT,NoDDstar,FinalSel,isKenriched,isIsolated,isLcpipi
0,True,False,False,True,True,False,2293.895486,0.139599,0.240121,-6.083391,...,False,True,False,0.001160,False,False,False,True,False,False
1,False,False,True,True,True,False,2241.863511,0.630536,0.040897,7.032762,...,True,True,False,0.273441,False,True,False,False,True,False
2,False,False,True,True,True,False,2272.953145,0.056319,0.993643,9.509515,...,True,True,False,0.011024,False,False,False,False,False,False
3,True,False,True,True,True,False,2249.818104,0.021579,0.999310,2.282870,...,True,True,True,0.008728,False,False,False,False,False,False
4,False,False,True,True,True,False,2320.249556,0.495502,0.988775,5.231485,...,True,True,False,0.223288,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,True,False,True,True,True,False,2271.636965,0.042828,0.998249,5.800219,...,True,True,True,0.011298,False,False,False,True,False,False
499996,False,False,False,False,True,False,2328.887717,0.029709,0.090516,5.794516,...,True,True,False,0.005908,False,False,False,False,False,False
499997,False,False,False,True,True,False,2273.122082,0.014471,0.025393,-1.795917,...,False,False,False,0.003769,False,False,False,False,True,False
499998,True,False,False,False,True,False,2233.107924,0.285419,0.771195,4.822522,...,True,True,False,0.001157,False,False,False,False,False,False


In [19]:
ComputeSweights('Preselected_Kenr.root','DataSS','MagUp','Kenriched')

'Preselected_Kenr_sw.root'

[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooAddPdf::model
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooAddPdf::sig
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooGaussian::sig1
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooRealVar::Lc_M
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooRealVar::mean
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooRealVar::sigma1
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooGaussian::sig2
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooRealVar::sigma2
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooRealVar::sig1frac
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooRealVar::nsig
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooExponential::bkg
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooRealVar::l
[#1] INFO:ObjectHandling -- RooWorkspace::import() impor

Info in <TCanvas::Print>: png file FitResults/Fit_DataSS_MagUp_Kenr.png has been created


In [20]:
f_Kenr_sw = uproot.open('Preselected_Kenr_sw.root')
t_Kenr_sw = f_Kenr_sw['DecayTree']
df_Kenr_sw = t_Kenr_sw.pandas.df()
df_Kenr_sw = df_Kenr_sw.set_index('__index__')
df_Kenr_sw

,Lb_L0Global_TIS,Lb_L0HadronDecision_TOS,Lc_Hlt1TrackMVADecision_TOS,Lc_Hlt1TwoTrackMVADecision_TOS,Lb_Hlt2XcMuXForTauB2XcMuDecision_TOS,Lb_Hlt2XcMuXForTauB2XcFakeMuDecision_TOS,Lc_M,p_ProbNNp,p_ProbNNk,mu_PIDmu,...,PIDCalib,Preselection,bdt,PassBDT,NoDDstar,FinalSel,isKenriched,isIsolated,sw_bkg,sw_sig
__index__,,,,,,,,,,,,,,,,,,,,,
1853,True,True,True,True,True,False,2250.095442,0.985684,0.003054,7.431428,...,True,True,0.740548,True,True,True,True,False,1.391109,-0.390972
3952,True,False,True,True,True,False,2285.572467,0.998481,0.002962,2.964477,...,True,True,0.825305,True,True,True,True,False,-0.132278,1.132267
6234,True,False,False,True,True,False,2238.172357,0.999352,0.002602,11.341412,...,True,True,0.947886,True,True,True,True,False,1.391111,-0.390974
8297,True,False,True,True,True,False,2316.789265,0.928981,0.752900,11.664609,...,True,True,0.853298,True,True,True,True,False,1.389646,-0.389509
8330,True,True,True,True,True,False,2262.039345,0.964841,0.000413,7.072842,...,True,True,0.901033,True,True,True,True,False,1.385262,-0.385125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491795,True,False,True,True,True,True,2313.833507,0.998406,0.005418,3.971451,...,True,True,0.890014,True,True,True,True,False,1.383387,-0.383251
493063,True,False,False,True,True,False,2274.124007,0.937879,0.921948,3.518419,...,True,True,0.824423,True,True,True,True,False,0.766502,0.233574
494929,False,True,True,False,True,False,2312.218523,0.992694,0.027039,5.609961,...,True,True,0.920091,True,True,True,True,False,1.373428,-0.373293


In [27]:
%%time
#Create Iso dataframe
dfIso =  dfT.loc[(dfT.FinalSel==True) & (dfT.isIsolated==True) & (dfT.isKenriched==False)].copy()
#dfIso = dfIso.drop(dfIso.loc[:,'L0':'isIsolated'].columns,axis=1)
dfIso.to_root('Preselected_Iso.root','DecayTree')
ComputeSweights('Preselected_Iso.root','DataSS','MagUp','iso')
f_Iso_sw = uproot.open('Preselected_Iso_sw.root')
t_Iso_sw = f_Iso_sw['DecayTree']
df_Iso_sw = t_Iso_sw.pandas.df()
df_Iso_sw = df_Iso_sw.set_index('__index__')

CPU times: user 347 ms, sys: 21.5 ms, total: 368 ms
Wall time: 420 ms
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooAddPdf::model
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooAddPdf::sig
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooGaussian::sig1
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooRealVar::Lc_M
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooRealVar::mean
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooRealVar::sigma1
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooGaussian::sig2
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooRealVar::sigma2
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooRealVar::sig1frac
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooRealVar::nsig
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooExponential::bkg
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing 

Info in <TCanvas::Print>: png file FitResults/Fit_DataSS_MagUp_iso.png has been created


In [28]:
df_Iso_sw

,Lb_L0Global_TIS,Lb_L0HadronDecision_TOS,Lc_Hlt1TrackMVADecision_TOS,Lc_Hlt1TwoTrackMVADecision_TOS,Lb_Hlt2XcMuXForTauB2XcMuDecision_TOS,Lb_Hlt2XcMuXForTauB2XcFakeMuDecision_TOS,Lc_M,p_ProbNNp,p_ProbNNk,mu_PIDmu,...,PIDCalib,Preselection,bdt,PassBDT,NoDDstar,FinalSel,isKenriched,isIsolated,sw_bkg,sw_sig
__index__,,,,,,,,,,,,,,,,,,,,,
202,False,True,False,True,True,False,2290.436312,0.995413,0.097469,10.713131,...,True,True,0.966301,True,True,True,False,True,-0.117790,1.117781
1274,True,False,True,True,True,False,2291.481180,0.898340,0.053723,6.927545,...,True,True,0.730956,True,True,True,False,True,-0.092806,1.092798
1297,True,False,False,True,True,False,2241.032240,0.998536,0.002199,4.987059,...,True,True,0.928488,True,True,True,False,True,1.479076,-0.479063
1320,False,True,True,True,True,False,2320.958530,0.933802,0.002500,5.212759,...,True,True,0.913888,True,True,True,False,True,1.477639,-0.477626
1556,True,True,True,False,True,False,2292.701463,0.999783,0.001238,10.278585,...,True,True,0.951211,True,True,True,False,True,-0.051433,1.051425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499051,True,False,True,True,True,False,2284.949671,0.977857,0.083864,8.781097,...,True,True,0.941365,True,True,True,False,True,-0.136164,1.136155
499147,True,False,True,True,True,False,2232.897445,0.861627,0.228485,3.229308,...,True,True,0.700924,True,True,True,False,True,1.479077,-0.479064
499196,True,True,True,True,True,False,2279.924122,0.941900,0.000393,4.221556,...,True,True,0.883996,True,True,True,False,True,0.025739,0.974254


In [30]:
%%time
#create Full dataframe
dfFull =  dfT.loc[(dfT.FinalSel==True)].copy()
#dfFull = dfFull.drop(dfFull.loc[:,'L0':'isIsolated'].columns,axis=1)
dfFull.to_root('Preselected_Full.root','DecayTree')
ComputeSweights('Preselected_Full.root','DataSS','MagUp','full')
f_Full_sw = uproot.open('Preselected_Full_sw.root')
t_Full_sw = f_Full_sw['DecayTree']
df_Full_sw = t_Full_sw.pandas.df()
df_Full_sw = df_Full_sw.set_index('__index__')

CPU times: user 866 ms, sys: 31.5 ms, total: 897 ms
Wall time: 1.01 s
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooAddPdf::model
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooAddPdf::sig
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooGaussian::sig1
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooRealVar::Lc_M
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooRealVar::mean
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooRealVar::sigma1
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooGaussian::sig2
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooRealVar::sigma2
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooRealVar::sig1frac
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooRealVar::nsig
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooExponential::bkg
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing 

Info in <TCanvas::Print>: png file FitResults/Fit_DataSS_MagUp.png has been created


In [54]:
dfFinal = dfT.merge(df_Full_sw[['sw_sig','sw_bkg']],how='left',left_index=True,right_index=True)
dfFinal = dfFinal.rename(columns={'sw_sig':'sw_sig_Full','sw_bkg':'sw_bkg_Full'})
dfFinal

,Lb_L0Global_TIS,Lb_L0HadronDecision_TOS,Lc_Hlt1TrackMVADecision_TOS,Lc_Hlt1TwoTrackMVADecision_TOS,Lb_Hlt2XcMuXForTauB2XcMuDecision_TOS,Lb_Hlt2XcMuXForTauB2XcFakeMuDecision_TOS,Lc_M,p_ProbNNp,p_ProbNNk,mu_PIDmu,...,PIDCalib,Preselection,bdt,PassBDT,NoDDstar,FinalSel,isKenriched,isIsolated,sw_sig_Full,sw_bkg_Full
0,True,False,False,True,True,False,2293.895486,0.139599,0.240121,-6.083391,...,True,False,0.001160,False,False,False,True,False,NaN,NaN
1,False,False,True,True,True,False,2241.863511,0.630536,0.040897,7.032762,...,True,False,0.273441,False,True,False,False,True,NaN,NaN
2,False,False,True,True,True,False,2272.953145,0.056319,0.993643,9.509515,...,True,False,0.011024,False,False,False,False,False,NaN,NaN
3,True,False,True,True,True,False,2249.818104,0.021579,0.999310,2.282870,...,True,True,0.008728,False,False,False,False,False,NaN,NaN
4,False,False,True,True,True,False,2320.249556,0.495502,0.988775,5.231485,...,True,False,0.223288,False,False,False,False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,True,False,True,True,True,False,2271.636965,0.042828,0.998249,5.800219,...,True,True,0.011298,False,False,False,True,False,NaN,NaN
499996,False,False,False,False,True,False,2328.887717,0.029709,0.090516,5.794516,...,True,False,0.005908,False,False,False,False,False,NaN,NaN
499997,False,False,False,True,True,False,2273.122082,0.014471,0.025393,-1.795917,...,False,False,0.003769,False,False,False,False,True,NaN,NaN
499998,True,False,False,False,True,False,2233.107924,0.285419,0.771195,4.822522,...,True,False,0.001157,False,False,False,False,False,NaN,NaN


In [56]:
dfFinal = dfFinal.merge(df_Iso_sw[['sw_sig','sw_bkg']],how='left',left_index=True,right_index=True)
dfFinal

,Lb_L0Global_TIS,Lb_L0HadronDecision_TOS,Lc_Hlt1TrackMVADecision_TOS,Lc_Hlt1TwoTrackMVADecision_TOS,Lb_Hlt2XcMuXForTauB2XcMuDecision_TOS,Lb_Hlt2XcMuXForTauB2XcFakeMuDecision_TOS,Lc_M,p_ProbNNp,p_ProbNNk,mu_PIDmu,...,bdt,PassBDT,NoDDstar,FinalSel,isKenriched,isIsolated,sw_sig_Full,sw_bkg_Full,sw_sig,sw_bkg
0,True,False,False,True,True,False,2293.895486,0.139599,0.240121,-6.083391,...,0.001160,False,False,False,True,False,NaN,NaN,NaN,NaN
1,False,False,True,True,True,False,2241.863511,0.630536,0.040897,7.032762,...,0.273441,False,True,False,False,True,NaN,NaN,NaN,NaN
2,False,False,True,True,True,False,2272.953145,0.056319,0.993643,9.509515,...,0.011024,False,False,False,False,False,NaN,NaN,NaN,NaN
3,True,False,True,True,True,False,2249.818104,0.021579,0.999310,2.282870,...,0.008728,False,False,False,False,False,NaN,NaN,NaN,NaN
4,False,False,True,True,True,False,2320.249556,0.495502,0.988775,5.231485,...,0.223288,False,False,False,False,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,True,False,True,True,True,False,2271.636965,0.042828,0.998249,5.800219,...,0.011298,False,False,False,True,False,NaN,NaN,NaN,NaN
499996,False,False,False,False,True,False,2328.887717,0.029709,0.090516,5.794516,...,0.005908,False,False,False,False,False,NaN,NaN,NaN,NaN
499997,False,False,False,True,True,False,2273.122082,0.014471,0.025393,-1.795917,...,0.003769,False,False,False,False,True,NaN,NaN,NaN,NaN
499998,True,False,False,False,True,False,2233.107924,0.285419,0.771195,4.822522,...,0.001157,False,False,False,False,False,NaN,NaN,NaN,NaN


In [58]:
dfFinal = dfFinal.rename(columns={'sw_sig':'sw_sig_Iso','sw_bkg':'sw_bkg_Iso'})
dfFinal
dfFinal[(dfFinal.FinalSel==True) & (dfFinal.isKenriched==True)]

,Lb_L0Global_TIS,Lb_L0HadronDecision_TOS,Lc_Hlt1TrackMVADecision_TOS,Lc_Hlt1TwoTrackMVADecision_TOS,Lb_Hlt2XcMuXForTauB2XcMuDecision_TOS,Lb_Hlt2XcMuXForTauB2XcFakeMuDecision_TOS,Lc_M,p_ProbNNp,p_ProbNNk,mu_PIDmu,...,bdt,PassBDT,NoDDstar,FinalSel,isKenriched,isIsolated,sw_sig_Full,sw_bkg_Full,sw_sig_Iso,sw_bkg_Iso
1853,True,True,True,True,True,False,2250.095442,0.985684,0.003054,7.431428,...,0.740548,True,True,True,True,False,-0.410529,1.410530,NaN,NaN
3952,True,False,True,True,True,False,2285.572467,0.998481,0.002962,2.964477,...,0.825305,True,True,True,True,False,1.163837,-0.163838,NaN,NaN
6234,True,False,False,True,True,False,2238.172357,0.999352,0.002602,11.341412,...,0.947886,True,True,True,True,False,-0.410663,1.410665,NaN,NaN
8297,True,False,True,True,True,False,2316.789265,0.928981,0.752900,11.664609,...,0.853298,True,True,True,True,False,-0.405217,1.405218,NaN,NaN
8330,True,True,True,True,True,False,2262.039345,0.964841,0.000413,7.072842,...,0.901033,True,True,True,True,False,-0.375514,1.375515,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491795,True,False,True,True,True,True,2313.833507,0.998406,0.005418,3.971451,...,0.890014,True,True,True,True,False,-0.391728,1.391729,NaN,NaN
493063,True,False,False,True,True,False,2274.124007,0.937879,0.921948,3.518419,...,0.824423,True,True,True,True,False,0.356570,0.643430,NaN,NaN
494929,False,True,True,False,True,False,2312.218523,0.992694,0.027039,5.609961,...,0.920091,True,True,True,True,False,-0.375456,1.375457,NaN,NaN
495056,False,True,False,True,True,True,2284.763664,0.882167,0.299303,3.319171,...,0.842820,True,True,True,True,False,1.153151,-0.153151,NaN,NaN


In [62]:
dfFinal = dfFinal.merge(df_Kenr_sw[['sw_sig','sw_bkg']],how='left',left_index=True,right_index=True)
dfFinal = dfFinal.rename(columns={'sw_sig':'sw_sig_Kenr','sw_bkg':'sw_bkg_Kenr'})

In [63]:
dfFinal

,Lb_L0Global_TIS,Lb_L0HadronDecision_TOS,Lc_Hlt1TrackMVADecision_TOS,Lc_Hlt1TwoTrackMVADecision_TOS,Lb_Hlt2XcMuXForTauB2XcMuDecision_TOS,Lb_Hlt2XcMuXForTauB2XcFakeMuDecision_TOS,Lc_M,p_ProbNNp,p_ProbNNk,mu_PIDmu,...,sw_sig_Full,sw_bkg_Full,sw_sig_Iso,sw_bkg_Iso,sw_sig_x,sw_bkg_x,sw_sig_y,sw_bkg_y,sw_sig_Kenr,sw_bkg_Kenr
0,True,False,False,True,True,False,2293.895486,0.139599,0.240121,-6.083391,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,False,True,True,True,False,2241.863511,0.630536,0.040897,7.032762,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,False,True,True,True,False,2272.953145,0.056319,0.993643,9.509515,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,True,False,True,True,True,False,2249.818104,0.021579,0.999310,2.282870,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,False,True,True,True,False,2320.249556,0.495502,0.988775,5.231485,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,True,False,True,True,True,False,2271.636965,0.042828,0.998249,5.800219,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499996,False,False,False,False,True,False,2328.887717,0.029709,0.090516,5.794516,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499997,False,False,False,True,True,False,2273.122082,0.014471,0.025393,-1.795917,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499998,True,False,False,False,True,False,2233.107924,0.285419,0.771195,4.822522,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
